# Library imports, configurations

In [2]:
import os
import re
import s3fs
import pandas as pd
import requests
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
from unidecode import unidecode
from wordcloud import WordCloud
import nltk
nltk.download('punkt')

from stop_words import get_stop_words
import unicodedata

from nltk.tokenize import word_tokenize
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from elasticsearch_dsl import connections
from bs4 import BeautifulSoup as bs

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [4]:
BUCKET = "jplaton/diffusion"
fs.ls(BUCKET)

['jplaton/diffusion/.keep',
 'jplaton/diffusion/ted',
 'jplaton/diffusion/visio_mel']

In [5]:
BUCKET_CLEM = "clementg/diffusion"
fs.ls(BUCKET_CLEM)

['clementg/diffusion/.keep',
 'clementg/diffusion/Fiche de Poste DINUM - francetransfert-3595970061- pdf.csv']

# Functions

In [86]:
# Suppression des accents

def remove_accent(s):

    s = unidecode(str(s))
    return(s)

# Suppression de la ponctuation

def remove_punctuation(s):
    # On retire la ponctuation
 
    s = re.sub(r'[^A-Za-z0-9]',' ',str(s))
    return(s)

# Réduction de la casse

def tolower(s):
    s = ' '.join(str(s).lower().split())
    return(s)


# Sur cette base, on crée une petite fonction qui retire les stop words
stopwords = get_stop_words('french')   
stopwords = [unicodedata.normalize('NFKD',m).encode('ASCII', 'ignore').decode() for m in stopwords]

def remove_stopwords(s):
    s = [w for w in word_tokenize(s) if w not in stopwords]
    s = ' '.join(s)
    return(s)

def remove_accent_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].map(remove_accent)
    return df_c

def remove_punctuation_from_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].map(remove_punctuation)
    return df_c

def tolower_df(df,list_cols):
    df_c =  df.copy()
    df_c[list_cols] = df_c[list_cols].map(tolower)
    return df_c

# Scraping de annuaire service public pour aller chercher les services à aprtir des ids
def get_service_from_scraping(service_id):
    try:
        url=f'https://lannuaire.service-public.fr/gouvernement/{service_id}'
        response = requests.get(url)
        html = response.content
        soup = bs(html, "lxml")
        return soup.title.get_text().replace(' - Annuaire | Service-public.fr', '')
    except:
        return ''

# DATA Download

In [7]:
FILE_PATH_TED = "ted"
FILE_PATH_TED_S3 = BUCKET + "/" + FILE_PATH_TED

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20230409.csv", mode="rb") as file_in:
    offres_2023_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/offres-datagouv-20221225.csv", mode="rb") as file_in:   
    offres_2022_df = pd.read_csv(file_in, sep=";")

with fs.open(FILE_PATH_TED_S3+"/Bulloterie_10_Saison_2.xlsx", mode="rb") as file_in:  
    bulloterie_df =  pd.read_excel(file_in, sheet_name=0,header=[0,1])


/tmp/ipykernel_69563/4280061390.py:5: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2023_df = pd.read_csv(file_in, sep=";")
/tmp/ipykernel_69563/4280061390.py:8: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  offres_2022_df = pd.read_csv(file_in, sep=";")


In [8]:
with fs.open(BUCKET_CLEM+"/Fiche de Poste DINUM - francetransfert-3595970061- pdf.csv", mode="rb") as file_in:
    fdp_dinum_df = pd.read_csv(file_in, sep=";",encoding='Windows-1252')

In [9]:
with fs.open(FILE_PATH_TED_S3+"/dila_refOrga_admin_Etat_fr_20230505.json", mode="rb") as file_in:  
    orga =  json.load(file_in)

### Traitement référentiel services

In [87]:
orga_df = pd.json_normalize(orga,record_path='service')

In [93]:
orga_df['services_fils'] = orga_df['hierarchie'].map(lambda x: [get_service_from_scraping(x[i]['service']) for i in range(len(x))])

In [94]:
services_clean = ( orga_df
    .pipe(remove_accent_from_df,['nom','type_organisme','sigle','services_fils'])
    .pipe(remove_punctuation_from_df,['nom','type_organisme','sigle','services_fils'])
    .pipe(tolower_df,['nom','type_organisme','sigle','services_fils'])
)

In [95]:
services_clean = services_clean[['nom','type_organisme','sigle']] #,'services_fils' #services_clean[services_clean['services_fils'].map(len)==1]

In [96]:
services_clean.head(5)

,nom,type_organisme,sigle
0,bureau de coordination strategique,administration centrale ou ministere,
1,direction de l information legale et administrative,administration centrale ou ministere,dila
2,bureau de la simplification et de la qualite de service,administration centrale ou ministere,
3,service de la protection sdlp,administration centrale ou ministere,
4,service national d accueil telephonique de l enfance en danger snated,groupement d interet public,


### Traitement offres

In [64]:
offres_df = pd.concat([offres_2023_df,offres_2022_df])

In [65]:
offres_df.columns = offres_df.columns.map(remove_accent).map(remove_punctuation).map(tolower).map(lambda s: s.replace(' ', '_'))

 offres_df[offres_df['metier'].str.contains('scientist|engineer',na=False)].head(2)

In [66]:
offres_df_clean = ( offres_df
    .pipe(remove_accent_from_df,['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste'])
    .pipe(remove_punctuation_from_df,['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste'])
    .pipe(tolower_df,['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste'])
)

/tmp/ipykernel_69563/463726462.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(remove_accent)
/tmp/ipykernel_69563/463726462.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(remove_punctuation)
/tmp/ipykernel_69563/463726462.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(tolower)


In [67]:
offres_df_clean = offres_df_clean[['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste']] #offres_df_clean[offres_df_clean['metier'].str.contains("ux designer",na=False)|offres_df_clean['intitule_du_poste'].str.contains("data designer",na=False)].head(5)

### Traitement Fiche de poste DNUM

In [68]:
# Traitement des colonnes
fdp_dinum_df.columns = fdp_dinum_df.columns.map(remove_accent).map(remove_punctuation).map(tolower).map(lambda s: s.replace(' ', '_'))

In [69]:
fdp_clean = ( fdp_dinum_df
    .pipe(remove_accent_from_df,['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences'])
    .pipe(remove_punctuation_from_df,['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences'])
    .pipe(tolower_df,['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences'])
)

/tmp/ipykernel_69563/463726462.py:34: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(remove_accent)
/tmp/ipykernel_69563/463726462.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(remove_punctuation)
/tmp/ipykernel_69563/463726462.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_c[list_cols] = df_c[list_cols].applymap(tolower)


In [70]:
fdp_clean = fdp_clean[['poste','intitule_du_service_demandeur_bureau_section','corps_grade','presentation','missions','competences']] #fdp_clean.head(2)


# Insertion des données dans Elastic Search

In [71]:
# Paramétrage du client
es_client = connections.create_connection(hosts=['http://elasticsearch-master:9200/'])

In [72]:
# Fontion qui transforme un dataframe en document/index à stocker dans ElasticSearch
def doc_generator(df,index_name):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": index_name,
                "_type": "_doc",
                "_source": document,
            }

In [68]:
# Indexation des offres
helpers.bulk(es_client, doc_generator(offres_df_clean,"offres"))

/tmp/ipykernel_52094/3268320283.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es_client, doc_generator(offres_df_clean))
/tmp/ipykernel_52094/3268320283.py:1: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  helpers.bulk(es_client, doc_generator(offres_df_clean))


(406755, [])

In [73]:
# Indexation des services
helpers.bulk(es_client, doc_generator(services_clean,"services"))

/tmp/ipykernel_69563/1014443642.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es_client, doc_generator(services_clean,"services"))
/tmp/ipykernel_69563/1014443642.py:2: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  helpers.bulk(es_client, doc_generator(services_clean,"services"))


(7636, [])

In [89]:
# Indexation des fiches de poste
helpers.bulk(es_client, doc_generator(fdp_clean,"fiches"))

/tmp/ipykernel_72898/2169517871.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(es_client, doc_generator(fdp_clean,"fiches"))
/tmp/ipykernel_72898/2169517871.py:2: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  helpers.bulk(es_client, doc_generator(fdp_clean,"fiches"))


(30, [])

In [130]:
from elasticsearch_dsl import Search
s = Search(index='offres').query('match',metier='charge')

In [131]:
r=s.execute()

/opt/mamba/lib/python3.11/site-packages/elasticsearch_dsl/search.py:712: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.search(index=self._index, body=self.to_dict(), **self._params).body,


In [132]:
df_results = pd.DataFrame((d.to_dict() for d in s.scan()))

/opt/mamba/lib/python3.11/site-packages/elasticsearch_dsl/search.py:728: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  for hit in scan(es, query=self.to_dict(), index=self._index, **self._params):


In [133]:
df_results

,organisme_de_rattachement,metier,intitule_du_poste,localisation_du_poste
0,universite paris saclay,chargee charge de controle interne budgetaire et comptable,gestionnaire des operations financieres h f,essonne
1,structures de cooperation territoriale,chargee charge de developpement territorial,che fe de projet petites villes de demain com com du civraisien en poitou,vienne
2,centres communaux d action sociale,chargee charge de gestion administrative et ou de paie,assistant charge du portage de repas et du logement h f,morbihan
3,conseils departementaux,chargee charge de gestion administrative et ou de paie,secretaire du service crip conseil departemental de la gironde,gironde
4,communes,chargee charge d amenagement des espaces verts et de production vegetale,agent d entretien des espaces verts h f le petit quevilly,seine maritime
...,...,...,...,...
153856,communes,chargee charge de l instruction de dossiers administratifs,instructeur instructrice des demandes autorisations droits des sols commune d aix en provence,bouches du rhone
153857,communes,chargee charge des d exploitation des equipements sportifs et ludiques,chef d equipe exploitation h f lyon,rhone
153858,direction des achats de l etat dae,chargee charge de conduite des politiques publiques,dae consultant interne etudes actions d optimisation des achats de l etat et de ses operateurs h f,paris
153859,communes,chargee charge de l instruction de dossiers administratifs,agent des formalites civiles commune d aix en provence,bouches du rhone


In [96]:
 # Recherche dans l'ensemble des champs le meilleur écho (le plus pertinent) #
fullsearch_easy = es_client.search(index = "services", # l'index dans lequel on cherche
                       q = "direction generale des finances publiques dgfip	data scientis ssi dpn dtnum bureau si mission service a l utilisateur data scientist expert h f", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités

/tmp/ipykernel_72898/3955335858.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch_easy = es_client.search(index = "services", # l'index dans lequel on cherche


In [97]:
fullsearch_easy

ObjectApiResponse({'took': 16, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 7636, 'relation': 'eq'}, 'max_score': 26.99309, 'hits': [{'_index': 'services', '_type': '_doc', '_id': 'bbFaKIsB1My3OOFZFDtC', '_score': 26.99309, '_source': {'copyright': 'direction de l information legale et administrative premiere ministre', 'nom': 'direction generale des finances publiques dgfip', 'type_organisme': 'administration centrale ou ministere', 'sigle': ''}}]}})

In [100]:
 # Recherche dans l'ensemble des champs le meilleur écho (le plus pertinent) #
fullsearch_scientist = es_client.search(index = "services", # l'index dans lequel on cherche
                       q = "centre hospitalier universitaire de tours	data scientist datascientist centre de donnees cliniques", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités

/tmp/ipykernel_72898/2210460514.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch1 = es_client.search(index = "services", # l'index dans lequel on cherche


In [101]:
fullsearch_scientist

ObjectApiResponse({'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 7636, 'relation': 'eq'}, 'max_score': 15.201317, 'hits': [{'_index': 'services', '_type': '_doc', '_id': 'DrFaKIsB1My3OOFZEzmY', '_score': 15.201317, '_source': {'copyright': 'direction de l information legale et administrative premiere ministre', 'nom': 'centre de valorisation des ressources humaines cvrh de tours', 'type_organisme': 'service a competence nationale', 'sigle': ''}}]}})

In [103]:
 # Recherche dans l'ensemble des champs le meilleur écho (le plus pertinent) #
fullsearch_engineer= es_client.search(index = "services", # l'index dans lequel on cherche
                       q = "service hydrographique et oceanographique de la marine	data engineer	ingenieur en traitement de l information conception developpement et experimentation d ia h f", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités

/tmp/ipykernel_72898/1096055904.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch_engineer= es_client.search(index = "services", # l'index dans lequel on cherche


In [104]:
fullsearch_engineer

ObjectApiResponse({'took': 23, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 7636, 'relation': 'eq'}, 'max_score': 27.117067, 'hits': [{'_index': 'services', '_type': '_doc', '_id': 't7FaKIsB1My3OOFZFT--', '_score': 27.117067, '_source': {'copyright': 'direction de l information legale et administrative premiere ministre', 'nom': 'service hydrographique et oceanographique de la marine shom', 'type_organisme': 'etablissement public', 'sigle': ''}}]}})

In [106]:
fullsearch_designer = es_client.search(index = "services", # l'index dans lequel on cherche
                       q = "caisse des depots et consignations	architecte technique expert conception et architecture de donnees d entreprise data designer h f	", # notre requête textuelle
                              size = 1) # taille de l'ensemble les échos souhaités

/tmp/ipykernel_72898/3294352241.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch_designer = es_client.search(index = "services", # l'index dans lequel on cherche


In [107]:
fullsearch_designer

ObjectApiResponse({'took': 17, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 7636, 'relation': 'eq'}, 'max_score': 21.853615, 'hits': [{'_index': 'services', '_type': '_doc', '_id': 'RbFaKIsB1My3OOFZEDBa', '_score': 21.853615, '_source': {'copyright': 'direction de l information legale et administrative premiere ministre', 'nom': 'caisse des depots et consignations cdc', 'type_organisme': 'juridiction', 'sigle': ''}}]}})

In [82]:
def get_services(es,query):
    fullsearch = es.search(index = "services", # l'index dans lequel on cherche
                       q = query, # notre requête textuelle
                       size = 1)
    return fullsearch['hits']['hits'][0]['_source']['services_fils']

In [83]:
def add_services(df, list_cols,es):
    df['service'] = df[list_cols].apply(lambda row: get_services(es,' '.join(row.values.astype(str))), axis=1)
    return df

In [119]:
get_services(es_client,"caisse des depots et consignations	architecte technique expert conception et architecture de donnees d entreprise data designer h f")

/tmp/ipykernel_72898/198050217.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch = es.search(index = "services", # l'index dans lequel on cherche


'caisse des depots et consignations cdc'

In [84]:
list_cols = ['organisme_de_rattachement','metier','intitule_du_poste','localisation_du_poste']
offres_df_temp = offres_df_clean[offres_df_clean['metier'].str.contains("data scientist",na=False)|offres_df_clean['intitule_du_poste'].str.contains("data scientist",na=False)]
offres_with_services =  offres_df_temp.head(50).pipe(add_services,list_cols,es_client)

/tmp/ipykernel_69563/1125244949.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch = es.search(index = "services", # l'index dans lequel on cherche
/tmp/ipykernel_69563/2305960098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['service'] = df[list_cols].apply(lambda row: get_services(es,' '.join(row.values.astype(str))), axis=1)


In [85]:
offres_with_services

,organisme_de_rattachement,metier,intitule_du_poste,localisation_du_poste,service
362,centre hospitalier universitaire de tours,data scientist,datascientist centre de donnees cliniques,nan,
7616,museum national d histoire naturelle,data scientist,ingenieur en calcul scientifique h f,paris 75,jardin des plantes musee de l homme parc zoologique de paris zoo de vincennes
8887,direction generale des finances publiques dgfip,data scientist,ssi dpn dtnum bureau si2 mission service a l utilisateur data scientist expert h f,seine saint denis 93,cabinet et communication cabinet international delegation aux relations internationales delegation a la transformation numerique dtnum mission de cooperation internationale mission strategie des relations avec les differents publics mission unification du recouvrement fiscal murf direction de l immobilier de l etat direction de la legislation fiscale service de la securite juridique et du controle fiscal sjcf service de la gestion fiscale gf service des gestions publiques locales des activites bancaires et economiques service de la fonction financiere et comptable de l etat 2fce service des ressources humaines rh service strategie pilotage budget spib service des systemes d information si service des retraites de l etat sre service d enquetes judiciaires des finances
8902,institut national de l information geographique et forestiere,data scientist,ingenieur conception consolidation et prototypage jumeaux numeriques,seine et marne 77,direction des programmes et de l appui aux politiques publiques dp direction des sciences et technologies de l information dsti direction des operations et des territoires dot
11965,ministere de l education nationale et de la jeunesse ministere de l enseignement superieur et de la recherche,data scientist,deep b2 2 data scientist responsable d analyses de donnees de panel f h,paris 75,cabinet du ministre de l education nationale et de la jeunesse bureau des cabinets de l education nationale et de la jeunesse de l enseignement superieur et de la recherche et des sports et des jeux olympiques et paralympiques ministre deleguee aupres du ministre du travail du plein emploi et de l insertion et du ministre de l education nationale et de la jeunesse chargee de l enseignement et de la formation professionnels secretaire d etat aupres du ministre des armees et du ministre de l education nationale et de la jeunesse chargee de la jeunesse et du service national universel secretariat general education nationale enseignement superieur recherche des sports et des jeux olympiques et paralympiques inspection generale de l education du sport et de la recherche igesr direction generale de l enseignement scolaire direction de la jeunesse de l education populaire et de la vie associative djepva delegue a la protection des donnees services centraux et organismes rattaches organismes sous tutelle de l education nationale delegue a l information et a l orientation secretariat general des ministeres charges des affaires sociales mission ministerielle d audit interne delegation generale a l emploi et a la formation professionnelle dgefp direction generale de la cohesion sociale service de la strategie des formations et de la vie etudiante dgesip a
15874,institut national de l information geographique et forestiere,data scientist,data scientist f h,val de marne 94,direction des programmes et de l appui aux politiques publiques dp direction des sciences et technologies de l information dsti direction des operations et des territoires dot
16651,secretariat general,chargee charge d etudes et d enquetes,charge d etudes specialise chaine penale data scientist,paris 75,departement exploitation charge de l instruction et du suivi des dossiers en lien avec les etias ou visa departement du soutien charge des ressources humaines du budget de la logistique et l immobilier des etudes et de la documentation ainsi que des systemes d information
16711,direction generale des finances publiques dgfip,data 

In [148]:
list_cols = ['poste','intitule_du_service_demandeur_bureau_section','corps_grade'] #,'presentation','missions','competences'
fdp_temp = fdp_clean[fdp_clean['poste'].str.contains("data engineer",na=False)|fdp_clean['competences'].str.contains("data engineer",na=False)]
fdp_with_services =  fdp_temp.pipe(add_services,list_cols,es_client)

/tmp/ipykernel_72898/198050217.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  fullsearch = es.search(index = "services", # l'index dans lequel on cherche
/tmp/ipykernel_72898/2305960098.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['service'] = df[list_cols].apply(lambda row: get_services(es,' '.join(row.values.astype(str))), axis=1)


In [149]:
fdp_with_services

,poste,intitule_du_service_demandeur_bureau_section,corps_grade,presentation,missions,competences,service
24,ingenieur data f h,direction interministerielle du numerique dinum departement etalab,,la direction interministerielle du numerique dinum est un service de la premiere ministre place sous l autorite du ministre de la transformation et de la fonction publiques elle oriente anime soutient et coordonne les actions des administrations de l etat et celles des organismes places sous sa tutelle visant a ameliorer la qualite l efficacite l efficience et la fiabilite des services rendus par le systeme d information et de communication de l etat elle contribue a l ouverture des donnees publiques aux strategies d innovation ouverte et au travail avec les ecosystemes d innovation elle pilote la politique de mutualisation du systeme d information et de communication de l etat et peut intervenir dans la gouvernance des operations de mutualisation ainsi que dans la conception l elaboration et la mise en oeuvre de ces operations enfin elle soutient le developpement des competences de l etat et des organismes places sous sa tutelle dans le domaine du numerique en facilitant l emergence d une filiere professionnelle interministerielle du numerique rejoindre la dinum c est prendre part a un collectif de plus de 200 agents reunis autour d une meme ambition le numerique au service de l efficacite de l action publique la le titulaire du poste integre le departement etalab de la dinum etalab vise a valoriser et exploiter les donnees comme levier d efficacite de l action publique il coordonne la conception et la mise en oeuvre de la strategie de l etat dans le domaine de la donnee il promeut l action de l etat en matiere d ouverture de circulation et d exploitation des donnees publiques il accompagne les administrations dans l usage de la data science de l intelligence artificielle et diffuse une culture de l innovation ouverte au sein des administrations,au sein du datalab l ingenieur e data travaille avec un product owner des data scientists et un analyste de donnees pour developper des cas d usages mobilisant des donnees et a fort impact au sein d une equipe produit suivant la methode agile l agent a pour mission de mettre en place des outils de traitement de la donnee pour realiser des cas d usage il elle met en oeuvre des traitements sur les donnees manipulees dans le cadre des cas d usage portes par le datalab afin de favoriser et de faciliter la reutilisation de celles ci par son equipe vous serez amene e a utiliser et mettre en oeuvre des outils de traitements et de stockage de donnees bases de donnees pour faciliter leur mise a jour et leur mise a disposition vous serez sensible a la question de la securite et de l administration systeme et gererez cette problematique dans les infrastructures data que vous utiliserez a ce titre vous collaborerez etroitement avec le responsable datasecops f h de l equipe faciliter l interoperabilite des donnees en integrant des variables pivots permettant le croisement avec d autres donnees et vous assurer du respect de bonnes pratiques techniques permettant une meilleure exploitation des donnees pour cela vous integrerez les dimensions de completude d unicite de conformite de tracabilite et de coconformite de tracabilite et de coherence des donnees garantir les bonnes pratiques de developpements appliquees sur les donnees traitements montee en qualite calculs d indicateurs dans l optique de maximiser l impact de projet data a fort potentiel veiller a ce que les outils et traitements developpes dans le cadre du datalab soient transferables aux entites chargees de les executer en production dans les administrations lorsque les cas d usage data ont demontre leur impact et sont finalises mener une veille technologique active pour mobiliser les meilleures technologies au cours de la realisation de vos missions vous devriez etre en veille permanente pour detecter des projets a impact a valoriser ou a accomp